In [ ]:
import os
import cv2
import numpy as np
import csv
from scipy.optimize import fmin

# Close all figures
cv2.destroyAllWindows()

# Define the folder paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"  # Replace with the path to the original image folder
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"  # Replace with the path to the output image folder
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"  # Replace with the desired path for the CSV file

# Get the list of files in the original image folder
original_files = os.listdir(original_folder)

# Create an empty list to store gamma values
gamma_values = []

# Iterate over the files and process each image
for file in original_files:
    # Read the original and output images
    original_image_path = os.path.join(original_folder, file)
    output_image_path = os.path.join(output_folder, file)
    
    # Load the original and output images
    original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
    output_image = cv2.imread(output_image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the images are successfully loaded
    if original_image is None or output_image is None:
        print(f'Failed to load image {file}. Please check the file path.')
        continue

    # Resize the images to 300x400 pixels
    original_image = cv2.resize(original_image, (400, 300))
    output_image = cv2.resize(output_image, (400, 300))

    # Convert images to double precision
    original_image = original_image.astype(np.float64)
    output_image = output_image.astype(np.float64)

    # Define the objective function to minimize
    def objectiveFunction(gamma):
        diff = original_image ** gamma - output_image
        return np.linalg.norm(diff, 'fro')

    # Perform optimization to find the optimal gamma value
    initialGuess = 0.6  # Initial guess for gamma
    gamma = fmin(objectiveFunction, initialGuess, disp=False)

    # Append the gamma value to the list
    gamma_values.append(gamma[0])

    # Display the optimal gamma value
    print(f'Image: {file} - Optimal gamma value: {gamma[0]:.4f}')

# Save gamma values to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Image', 'Gamma'])
    for file, gamma in zip(original_files, gamma_values):
        writer.writerow([file, gamma])

print(f"Gamma values saved to the CSV file: {csv_file_path}")

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate
from tensorflow.keras.optimizers import Adam

# Set file paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\SRFP DAILY REPORTS\SRFP FINAL REPORT\18901933200 (1).jpg"

# Load gamma values from the CSV file
gamma_df = pd.read_csv(csv_file_path)
gamma_dict = dict(zip(gamma_df['file_name'], gamma_df['gamma_value']))

# Define function to load, resize, and preprocess images
def load_images(original_folder, output_folder, file_name):
    original_path = os.path.join(original_folder, file_name)
    enhanced_path = os.path.join(output_folder, file_name)

    original_img = cv2.imread(original_path)
    enhanced_img = cv2.imread(enhanced_path)

    # Resize images
    original_img = cv2.resize(original_img, (500, 500))
    enhanced_img = cv2.resize(enhanced_img, (500, 500))

    original_img = original_img / 255.0  # Normalize original image
    enhanced_img = enhanced_img / 255.0  # Normalize enhanced image

    return original_img, enhanced_img

# Create the training dataset
input_images = []
output_images = []
gammas = []

for file_name in os.listdir(original_folder):
    original_img, enhanced_img = load_images(original_folder, output_folder, file_name)
    gamma = gamma_dict.get(file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

    input_images.append(original_img)
    output_images.append(enhanced_img)
    gammas.append(gamma)

input_images = np.array(input_images)
output_images = np.array(output_images)
gammas = np.array(gammas)

# Define the model architecture
def build_model():
    input_shape = (500, 500, 3)
    input_image = Input(shape=input_shape, name='input_image')
    input_gamma = Input(shape=(1,), name='input_gamma')

    conv1 = Conv2D(32, 3, activation='relu', padding='same')(input_image)
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv2)

    conv_gamma = tf.repeat(input_gamma, tf.shape(conv3)[1]*tf.shape(conv3)[2])

    conv_gamma = tf.reshape(conv_gamma, (-1, tf.shape(conv3)[1], tf.shape(conv3)[2], 1))

    concat = concatenate([conv3, conv_gamma], axis=-1)

    output = Conv2D(3, 3, activation='sigmoid', padding='same')(concat)

    model = Model(inputs=[input_image, input_gamma], outputs=output)
    model.compile(optimizer=Adam(lr=0.0001), loss='mse')

    return model

# Build and train the model
model = build_model()
model.fit([input_images, gammas], output_images, batch_size=32, epochs=10)

# Save the model weights
model.save_weights('model_weights.h5')

# Load the input image, resize, and preprocess it
input_img = cv2.imread(input_image_path)
input_img = cv2.resize(input_img, (500, 500))
input_img = input_img / 255.0  # Normalize input image
input_img = np.expand_dims(input_img, axis=0)  # Add batch dimension

# Get the optimal gamma value for the input image
input_file_name = os.path.basename(input_image_path)
optimal_gamma = gamma_dict.get(input_file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

# Generate the enhanced image
gamma_input = np.array([[optimal_gamma]])  # Convert gamma input to NumPy array
enhanced_img = model.predict([input_img, gamma_input])
enhanced_img = enhanced_img[0] * 255.0  # Denormalize enhanced image
enhanced_img = enhanced_img.astype(np.uint8)

# Save the enhanced image
output_image_path = os.path.join(output_folder, input_file_name)
cv2.imwrite(output_image_path, enhanced_img)


Epoch 1/10
10/10 [==============================] - 441s 38s/step - loss: 0.0769
Epoch 2/10
10/10 [==============================] - 448s 46s/step - loss: 0.0542
Epoch 3/10
10/10 [==============================] - 428s 40s/step - loss: 0.0418
Epoch 4/10
10/10 [==============================] - 341s 33s/step - loss: 0.0274
Epoch 5/10
10/10 [==============================] - 441s 43s/step - loss: 0.0128
Epoch 6/10
10/10 [==============================] - 366s 34s/step - loss: 0.0105
Epoch 7/10
10/10 [==============================] - 337s 32s/step - loss: 0.0114
Epoch 8/10
10/10 [==============================] - 426s 42s/step - loss: 0.0092
Epoch 9/10
10/10 [==============================] - 365s 35s/step - loss: 0.0083
Epoch 10/10
1/1 [==============================] - 0s 485ms/step


True

In [2]:
output_image_path

'C:\\Users\\MAHESH\\Desktop\\SRFP INTERNSHIP\\train_in_1\\Done\\18901933200 (1).jpg'

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import cv2
import os

# Set file paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\photo-.jpg"

# Load gamma values from the CSV file
gamma_df = pd.read_csv(csv_file_path)
gamma_dict = dict(zip(gamma_df['file_name'], gamma_df['gamma_value']))

# Define function to load, resize, and preprocess images
def load_images(original_folder, output_folder, file_name):
    original_path = os.path.join(original_folder, file_name)
    enhanced_path = os.path.join(output_folder, file_name)

    original_img = cv2.imread(original_path)
    enhanced_img = cv2.imread(enhanced_path)

    # Resize images
    original_img = cv2.resize(original_img, (250, 250))
    enhanced_img = cv2.resize(enhanced_img, (250, 250))

    original_img = original_img / 255.0  # Normalize original image
    enhanced_img = enhanced_img / 255.0  # Normalize enhanced image

    return original_img, enhanced_img

# Create the training dataset
input_images = []
output_images = []
gammas = []

for file_name in os.listdir(original_folder):
    original_img, enhanced_img = load_images(original_folder, output_folder, file_name)
    gamma = gamma_dict.get(file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

    input_images.append(original_img)
    output_images.append(enhanced_img)
    gammas.append(gamma)

input_images = np.array(input_images)
output_images = np.array(output_images)
gammas = np.array(gammas)

# Define the PyTorch model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv_gamma = nn.Conv2d(1, 32, kernel_size=1, stride=1)
        self.concat = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_image, input_gamma):
        conv1 = nn.ReLU()(self.conv1(input_image))
        conv2 = nn.ReLU()(self.conv2(conv1))
        conv3 = nn.ReLU()(self.conv3(conv2))

        conv_gamma = input_gamma.unsqueeze(2).unsqueeze(3)  # Adjust dimensions of conv_gamma

        concat = torch.cat((conv3, conv_gamma), dim=1)
        output = self.concat(concat)
        output = self.sigmoid(output)

        return output




# Build and train the PyTorch model
model = CustomModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

input_images_tensor = torch.from_numpy(input_images).float()
output_images_tensor = torch.from_numpy(output_images).float()
gammas_tensor = torch.from_numpy(gammas).float()

train_dataset = torch.utils.data.TensorDataset(input_images_tensor, gammas_tensor, output_images_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

epochs = 10
for epoch in range(epochs):
    for input_img, gamma, target_img in train_loader:
        optimizer.zero_grad()
        input_img = input_img.permute(0, 3, 1, 2)  # Adjust input image dimensions
        gamma = gamma.repeat(input_img.shape[0], 1)  # Repeat gamma values to match batch size
        output_img = model(input_img, gamma.unsqueeze(1))
        loss = criterion(output_img, target_img.permute(0, 3, 1, 2))  # Adjust target image dimensions
        loss.backward()
        optimizer.step()


# Save the model weights
torch.save(model.state_dict(), 'model_weights.pth')

# Load the input image, resize, and preprocess it
input_img = cv2.imread(input_image_path)
input_img = cv2.resize(input_img, (250, 250))
input_img = input_img / 255.0  # Normalize input image
input_img = np.expand_dims(input_img, axis=0)  # Add batch dimension
input_img_tensor = torch.from_numpy(input_img).float().permute(0, 3, 1, 2)  # Adjust input image dimensions

# Get the optimal gamma value for the input image
input_file_name = os.path.basename(input_image_path)
optimal_gamma = gamma_dict.get(input_file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary
gamma_input = torch.tensor([[optimal_gamma]]).float()

# Generate the enhanced image
with torch.no_grad():
    enhanced_img = model(input_img_tensor, gamma_input.unsqueeze(0)).squeeze()
enhanced_img = enhanced_img.numpy() * 255.0  # Denormalize enhanced image
enhanced_img = enhanced_img.astype(np.uint8)

# Save the enhanced image
output_image_path = os.path.join(output_folder, input_file_name)
cv2.imwrite(output_image_path, enhanced_img)

RuntimeError: Tensors must have same number of dimensions: got 4 and 5

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random, device_put
from jax import lax
from flax import linen as nn
from flax.training import train_state

# Set file paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG"

# Load gamma values from the CSV file
gamma_df = pd.read_csv(csv_file_path)
gamma_dict = dict(zip(gamma_df['file_name'], gamma_df['gamma_value']))

# Define function to load, resize, and preprocess images
def load_images(original_folder, output_folder, file_name):
    original_path = os.path.join(original_folder, file_name)
    enhanced_path = os.path.join(output_folder, file_name)

    original_img = cv2.imread(original_path)
    enhanced_img = cv2.imread(enhanced_path)

    # Resize images
    original_img = cv2.resize(original_img, (500, 500))
    enhanced_img = cv2.resize(enhanced_img, (500, 500))

    original_img = original_img / 255.0  # Normalize original image
    enhanced_img = enhanced_img / 255.0  # Normalize enhanced image

    return original_img, enhanced_img

# Create the training dataset
input_images = []
output_images = []
gammas = []

for file_name in os.listdir(original_folder):
    original_img, enhanced_img = load_images(original_folder, output_folder, file_name)
    gamma = gamma_dict.get(file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

    input_images.append(original_img)
    output_images.append(enhanced_img)
    gammas.append(gamma)

input_images = np.array(input_images)
output_images = np.array(output_images)
gammas = np.array(gammas)

# Define the model architecture
class CustomModel(nn.Module):
    conv1_filters = 32
    conv2_filters = 32
    conv3_filters = 32

    def setup(self):
        self.conv1 = nn.Conv(features=self.conv1_filters, kernel_size=(3, 3), padding="SAME")
        self.conv2 = nn.Conv(features=self.conv2_filters, kernel_size=(3, 3), padding="SAME")
        self.conv3 = nn.Conv(features=self.conv3_filters, kernel_size=(3, 3), padding="SAME")
        self.concat = nn.Conv(features=3, kernel_size=(3, 3), padding="SAME")
        self.sigmoid = nn.Activation(nn.sigmoid)

    def __call__(self, input_image, input_gamma):
        conv1 = nn.relu(self.conv1(input_image))
        conv2 = nn.relu(self.conv2(conv1))
        conv3 = nn.relu(self.conv3(conv2))

        conv_gamma = jnp.repeat(input_gamma, jnp.prod(conv3.shape[1:]))

        conv_gamma = jnp.reshape(conv_gamma, (-1, conv3.shape[1], conv3.shape[2], 1))

        concat = jnp.concatenate([conv3, conv_gamma], axis=-1)

        output = self.concat(concat)
        output = self.sigmoid(output)

        return output

# Build the model
rng = random.PRNGKey(0)
input_shape = (1, 500, 500, 3)
gamma_shape = (1, 1)
model = CustomModel()
params = model.init(rng, input_image=jnp.ones(input_shape), input_gamma=jnp.ones(gamma_shape))

# Define the loss function
def mse_loss(output, target):
    return jnp.mean(jnp.square(output - target))

# Define the optimizer
optimizer_def = flax.optim.Adam(learning_rate=0.001)
optimizer = optimizer_def.create(params)

# Training loop
@jax.jit
def train_step(optimizer, input_img, target_img, gamma):
    def loss_fn(model):
        output_img = model(input_img, gamma)
        return mse_loss(output_img, target_img)

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grad = grad_fn(optimizer.target)
    optimizer = optimizer.apply_gradient(grad)
    return optimizer, loss

num_epochs = 10
batch_size = 16

num_batches = len(input_images) // batch_size

for epoch in range(num_epochs):
    epoch_loss = 0.0

    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size

        input_batch = input_images[start_idx:end_idx]
        output_batch = output_images[start_idx:end_idx]
        gamma_batch = gammas[start_idx:end_idx]

        input_batch = jnp.array(input_batch)
        output_batch = jnp.array(output_batch)
        gamma_batch = jnp.array(gamma_batch)

        optimizer, loss = train_step(optimizer, input_batch, output_batch, gamma_batch)
        epoch_loss += loss

    epoch_loss /= num_batches
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss}")

# Save the trained model
model_state = optimizer.target
flax.serialization.save(model_state, r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479vv.JPG")

# Load the trained model
model_state = flax.serialization.load(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\path_to_saved_model")
model = CustomModel()
params = model_state.params
model = model.apply(params)


ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [ ]:
# Load the trained model
model_state = flax.serialization.load(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\path_to_saved_model")
model = CustomModel()
params = model_state.params
model = model.apply(params)

# Load the input image
input_img = cv2.imread(input_image_path)
input_img = cv2.resize(input_img, (500, 500))
input_img = input_img / 255.0

# Prepare the input for the model
input_img = jnp.expand_dims(input_img, axis=0)
gamma_value = 1.2  # Example gamma value

# Convert to JAX arrays
input_img = jnp.array(input_img)
gamma = jnp.array(gamma_value)

# Apply the model to enhance the image
enhanced_img = model(input_img, gamma)

# Convert the enhanced image back to NumPy array
enhanced_img = enhanced_img[0].numpy()
enhanced_img = (enhanced_img * 255.0).astype(np.uint8)

# Save the enhanced image
output_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479vv.JPG"
cv2.imwrite(output_image_path, enhanced_img)


In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.optimize import fmin
from torch.utils.data import Dataset, DataLoader

# Set file paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\photo-.jpg"

# Load gamma values from the CSV file
gamma_df = pd.read_csv(csv_file_path)
gamma_dict = dict(zip(gamma_df['file_name'], gamma_df['gamma_value']))

# Define function to load, resize, and preprocess images
def load_images(original_folder, output_folder, file_name):
    original_path = os.path.join(original_folder, file_name)
    enhanced_path = os.path.join(output_folder, file_name)

    original_img = cv2.imread(original_path)
    enhanced_img = cv2.imread(enhanced_path)

    # Resize images
    original_img = cv2.resize(original_img, (500, 500))
    enhanced_img = cv2.resize(enhanced_img, (500, 500))

    original_img = original_img / 255.0  # Normalize original image
    enhanced_img = enhanced_img / 255.0  # Normalize enhanced image

    return original_img, enhanced_img

# Create custom dataset class
class ImageDataset(Dataset):
    def __init__(self, original_folder, output_folder, gamma_dict):
        self.original_folder = original_folder
        self.output_folder = output_folder
        self.gamma_dict = gamma_dict
        self.file_names = os.listdir(original_folder)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, index):
        file_name = self.file_names[index]
        original_img, enhanced_img = load_images(self.original_folder, self.output_folder, file_name)
        gamma = self.gamma_dict.get(file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

        return torch.from_numpy(original_img).permute(2, 0, 1), torch.from_numpy(enhanced_img).permute(2, 0, 1), gamma

# Create the training dataset
dataset = ImageDataset(original_folder, output_folder, gamma_dict)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Define the model architecture
class ImageEnhancementModel(nn.Module):
    def __init__(self):
        super(ImageEnhancementModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv_gamma = nn.Conv2d(1, 32, 1)

        self.concat = nn.Conv2d(64, 3, 3, padding=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_image, input_gamma):
        conv1 = nn.ReLU()(self.conv1(input_image))
        conv2 = nn.ReLU()(self.conv2(conv1))
        conv3 = nn.ReLU()(self.conv3(conv2))

        conv_gamma = self.conv_gamma(input_gamma.repeat(1, conv3.size(2), conv3.size(3)).unsqueeze(1))

        concat = torch.cat((conv3, conv_gamma), dim=1)

        output = self.sigmoid(self.concat(concat))

        return output

# Build and train the model
model = ImageEnhancementModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        original_images, output_images, gammas = batch
        original_images = original_images.to(device)
        output_images = output_images.to(device)
        gammas = gammas.to(device)

        optimizer.zero_grad()
        enhanced_images = model(original_images, gammas)
        loss = criterion(enhanced_images, output_images)
        loss.backward()
        optimizer.step()

# Save the model weights
torch.save(model.state_dict(), 'model_weights.pt')

# Load the input image, resize, and preprocess it
input_img = cv2.imread(input_image_path)
input_img = cv2.resize(input_img, (500, 500))
input_img = input_img / 255.0  # Normalize input image
input_img = np.transpose(input_img, (2, 0, 1))  # Transpose image dimensions
input_img = torch.from_numpy(input_img).unsqueeze(0).to(device)  # Add batch dimension and move to device

# Get the optimal gamma value for the input image
input_file_name = os.path.basename(input_image_path)
optimal_gamma = gamma_dict.get(input_file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

# Generate the enhanced image
gamma_input = torch.tensor([[optimal_gamma]]).to(device)  # Convert gamma input to PyTorch tensor and move to device
enhanced_img = model(input_img, gamma_input).squeeze().detach().cpu().numpy() * 255.0  # Denormalize and move to CPU
enhanced_img = enhanced_img.transpose(1, 2, 0).astype(np.uint8)

# Save the enhanced image
output_image_path = os.path.join(output_folder, input_file_name)
cv2.imwrite(output_image_path, enhanced_img)


RuntimeError: Input type (double) and bias type (float) should be the same

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import pandas as pd
from PIL import Image

# Step 2: Define the dataset
class ImageDataset(Dataset):
    def __init__(self, csv_file, original_folder, enhanced_folder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.original_folder = original_folder
        self.enhanced_folder = enhanced_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_name = self.data.loc[idx, 'file_name']
        gamma_value = self.data.loc[idx, 'gamma_value']

        original_image_path = os.path.join(self.original_folder, file_name)
        enhanced_image_path = os.path.join(self.enhanced_folder, file_name)

        original_image = Image.open(original_image_path).convert('RGB')
        enhanced_image = Image.open(enhanced_image_path).convert('RGB')

        if self.transform:
            original_image = self.transform(original_image)
            enhanced_image = self.transform(enhanced_image)

        return original_image, enhanced_image, gamma_value

# Step 3: Define the CNN model
class ImageEnhancer(nn.Module):
    def __init__(self):
        super(ImageEnhancer, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 64 * 64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# Step 4: Prepare the data and model
csv_file = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
enhanced_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"

dataset = ImageDataset(csv_file, original_folder, enhanced_folder, transform=ToTensor())
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ImageEnhancer().to(device)

# Step 5: Define the training loop
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0

    for images_original, images_enhanced, gamma_values in dataloader:
        images_original = images_original.to(device)
        images_enhanced = images_enhanced.to(device)
        gamma_values = gamma_values.to(device)

        optimizer.zero_grad()

        outputs = model(images_original)
        loss = criterion(outputs, gamma_values.unsqueeze(1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Step 6: Enhance an image
def enhance_image(image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image_tensor = ToTensor()(image).unsqueeze(0).to(device)
    enhanced_tensor = model(image_tensor)
    enhanced_image = enhanced_tensor.squeeze(0).detach().cpu()

    # Convert the tensor back to an image
    enhanced_image = enhanced_image.permute(1, 2, 0).numpy()
    enhanced_image = (enhanced_image * 255).astype(np.uint8)

    return Image.fromarray(enhanced_image)

# Example usage
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\photo-.jpg"
enhanced_image = enhance_image(input_image_path)
enhanced_image.save(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\photooooo.jpg")


RuntimeError: stack expects each tensor to be equal size, but got [3, 4000, 6000] at entry 0 and [3, 2848, 4288] at entry 1